In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
import pathlib
pathlib.Path("data").mkdir(exist_ok=True)
pathlib.Path("assessments").mkdir(exist_ok=True)

In [4]:
TESTS = [
    ("max_pullups", "Max body-weight pull-ups (reps)"),
    ("weighted_pullups_5RM", "5-rep max weighted pull-ups (+lb)"),
    ("max_hang_5s", "Max weighted 5-sec hang (+lb)"),
    ("bw_hangs_failure", "Body-weight hang to failure (sec)"),
    ("repeater_20mm", "5 s on / 5 s off repeater to failure (total sec)")
]

In [5]:
import os
import datetime
import json

def record_assessment(username):
    os.makedirs("assessments", exist_ok=True)
    
    row = {
        "username": username,
        "date": datetime.date.today().isoformat(),
    }
    
    # Loop through the tests and prompt the user for a result
    for key, prompt in TESTS:
        value = float(input(f"{prompt}: "))
        row[key] = value
    
    # Write the user's assessment history to a file
    path = f"assessments/{username}.jsonl"
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(row) + "\n")

    print("Assessment saved.")

In [6]:
record_assessment("demo")

Assessment saved.


In [9]:
df = pd.read_json("assessments/demo.jsonl", lines=True)
df.head()

,username,date,max_pullups,weighted_pullups_5RM,max_hang_5s,bw_hangs_failure,repeater_20mm
0,demo,2025-06-19,10,25,100,60,300


In [ ]:
def get_grade_norms(grade, path="data/norms_boulder_lb.csv"):
    df = pd.read_csv(path)
    subset = df[df["grade"] == grade]
    # turn into {test: (low, high)}
    return dict(zip(subset["test"], zip(subset["low"], subset["high"])))

norms = get_grade_norms("V6") 
print(norms)

{'weighted_pullup_5RM': (25, 45), 'max_hang_5s': (45, 77)}


In [19]:
# Compares a climber's assessment results against grade norms
def score_against_norms(user_row: dict, norms: dict, z_threshold: float = 1.0):
    z_scores = {}
    flags = {}
    
    for test, (low, high) in norms.items():
        user_val = user_row.get(test)
        if user_val is None:
            continue

        bounds = np.asarray([low, high], dtype=float)
        mean = bounds.mean()
        std = np.ptp(bounds) / 2    # same as (max - min) / 2
        if std == 0:
            std = 1e-9  # avoid div-by-zero
            
        z = (user_val - mean) / std
        z_scores[test] = z     
        
        if z < -z_threshold:
            flags[test] = "weakness"
        elif z >  z_threshold:
            flags[test] = "strength"
        else:
            flags[test] = "average"

    return z_scores, flags

latest_row = (
    pd.read_json("assessments/demo.jsonl", lines=True)
      .iloc[-1]
      .to_dict()
)

In [20]:
grade = "V6"
norms = get_grade_norms(grade)

z, f = score_against_norms(latest_row, norms)

print("Z-scores: ", z)
print("Flags: ", f)

Z-scores:  {'max_hang_5s': np.float64(2.4375)}
Flags:  {'max_hang_5s': 'strength'}
